In [ ]:
import logging
import os
import json
import datetime

from data_diff import connect_to_table, diff_tables
from data_diff import TableDiffer
from itertools import islice

logging.basicConfig(level=logging.INFO)

In [ ]:
%env REDSHIFT_HOST= ''
%env REDSHIFT_DATABASE= ''
%env REDSHIFT_USER= ''
%env REDSHIFT_PASSWORD= ''
%env REDSHIFT_SCHEMA=''
%env REDSHIFT_TABLE=''
# %env POSTGRES_PASSWORD=
%env SNOWFLAKE_USER= ''
%env SNOWFLAKE_ACCOUNT= ''
%env SNOWFLAKE_ROLE= ''
%env SNOWFLAKE_WAREHOUSE= ''
%env SNOWFLAKE_DATABASE= ''
%env SNOWFLAKE_SCHEMA= ''
%env SNOWFLAKE_TABLE= ''

In [ ]:
snowflake_config = {"driver": "snowflake",
 "user": os.environ["SNOWFLAKE_USER"],
 "account": os.environ["SNOWFLAKE_ROLE"],
 "role": os.environ["SNOWFLAKE_ROLE"],
 "warehouse": os.environ["SNOWFLAKE_WAREHOUSE"],
 "database": os.environ["SNOWFLAKE_DATABASE"],
 "schema": os.environ["SNOWFLAKE_SCHEMA"],
 "authentcator":"externalbrowser"
}

redshift_config = {
    "driver": "redshift",
    "host": os.environ["REDSHIFT_HOST"],
    "database": os.environ["REDSHIFT_DATABASE"],
    "user": os.environ["REDSHIFT_USER"],
    "password": os.environ['REDSHIFT_PASSWORD'],
    "port":"5439"
}



In [ ]:
diff_tables?

In [ ]:
sf_table = connect_to_table(snowflake_config,os.environ['SNOWFLAKE_TABLE'], key_column="id", update_column="updated_at")

rs_table = connect_to_table(redshift_config, (os.environ["REDSHIFT_SCHEMA"], os.environ["REDSHIFT_TABLE"]), key_column="id", update_column="updated_at")

### How to understand the output?            
            ('-', columns) for items in table1 but not in table2
            ('+', columns) for items in table2 but not in table1

In [ ]:
DEFAULT_BISECTION_THRESHOLD = 1024 * 25
DEFAULT_BISECTION_FACTOR = 30

In [ ]:
differ = TableDiffer(
        bisection_factor=DEFAULT_BISECTION_FACTOR,
        bisection_threshold=DEFAULT_BISECTION_THRESHOLD,
        threaded=False,
        max_threadpool_size=2,
        debug=False,
    )

diff_iter = differ.diff_tables(sf_table, rs_table)
diff_iter = islice(diff_iter, int(10))

In [ ]:
diff = list(diff_iter)
unique_diff_count = len({i[0] for _, i in diff})
max_table_count = max(differ.stats["table1_count"], differ.stats["table2_count"])
percent = 100 * unique_diff_count / (max_table_count or 1)
plus = len([1 for op, _ in diff if op == "+"])
minus = len([1 for op, _ in diff if op == "-"])

json_output = {
    "different_rows": len(diff),
    "different_percent": percent,
    "different_+": plus,
    "different_-": minus,
    "total": max_table_count,
}
print(json.dumps(json_output))